In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

from sklearn.model_selection import KFold
from scipy import sparse as sps

Tensorflow is not available


c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val, data_test= split_data2(data, 0.2, validation_percentage=0.1)
data_train_val = data_train + data_val

In [3]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train_val, verbose=False)
candidate_recommender.fit()

[I 2024-01-02 22:14:09,507] A new study created in memory with name: P3Beta


In [4]:
n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12633,NaN
12634,NaN
12635,NaN


In [5]:
from tqdm import tqdm

cutoff = 50

for user_id in tqdm(range(n_users)):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe

100%|██████████| 12638/12638 [00:05<00:00, 2123.34it/s]


,ItemID
UserID,
0,"[15344, 20178, 15720, 8056, 2843, 674, 18491, ..."
1,"[9728, 12583, 2548, 5682, 18653, 46, 7961, 172..."
2,"[21303, 5777, 10028, 7070, 1354, 4808, 5136, 1..."
3,"[8073, 46, 589, 227, 6138, 14368, 812, 10611, ..."
4,"[12793, 19369, 19370, 17723, 12986, 18001, 129..."
...,...
12633,"[2809, 2817, 2814, 2807, 2815, 7069, 2818, 281..."
12634,"[11498, 3955, 3943, 3937, 2810, 3949, 17072, 1..."
12635,"[257, 472, 259, 3297, 248, 19417, 470, 8291, 6..."


In [6]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

,ItemID
UserID,
0,15344
0,20178
0,15720
0,8056
0,2843
...,...
12637,13251
12637,13252
12637,10900


In [7]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_test)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})

correct_recommendations

,UserID,ItemID
0,0,0
1,0,35
2,1,46
3,1,53
4,1,58
...,...,...
95741,12637,5342
95742,12637,8357
95743,12637,8797
95744,12637,16602


In [8]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)

training_dataframe

,UserID,ItemID,Label
0,0,15344,False
1,0,20178,False
2,0,15720,False
3,0,8056,False
4,0,2843,False
...,...,...,...
631895,12637,13251,False
631896,12637,13252,False
631897,12637,10900,False
631898,12637,10427,False


In [9]:
topPop = TopPop(data_train_val)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train_val)
p3alpha.fit()

easer_study = optuna.create_study(
    study_name="Easer",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train_val)
easer.fit()

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train_val)
itemknncf.fit()


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

[I 2024-01-02 22:14:25,882] A new study created in memory with name: P3Alpha


TopPopRecommender: URM Detected 217 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 115 ( 0.5%) items with no interactions.
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 115 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 3135.95 column/sec. Elapsed time 7.09 sec


[I 2024-01-02 22:14:34,489] A new study created in memory with name: Easer


EASE_R_Recommender: URM Detected 217 ( 1.7%) users with no interactions.
EASE_R_Recommender: URM Detected 115 ( 0.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 5.81 min


[I 2024-01-02 22:21:10,368] A new study created in memory with name: ItemKNNCF


ItemKNNCFRecommender: URM Detected 217 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 7789.21 column/sec. Elapsed time 2.85 sec


In [10]:
total_size = data_train.size + data_val.size + data_test.size
percentage_train = (data_train.size / total_size) * 100
percentage_val = (data_val.size / total_size) * 100
percentage_test = (data_test.size / total_size) * 100

print(f"Percentuale data_train: {percentage_train}%")
print(f"Percentuale data_val: {percentage_val}%")
print(f"Percentuale data_test: {percentage_test}%")

Percentuale data_train: 71.99987466839346%
Percentuale data_val: 8.000125331606542%
Percentuale data_test: 20.0%


In [11]:
import scipy.sparse as sps
from tqdm import tqdm

training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

training_dataframe

100%|██████████| 12638/12638 [00:37<00:00, 339.29it/s]


,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF
0,0,15344,False,4.0,0.029524,0.003029,0.019198
1,0,20178,False,3.0,0.010204,0.000170,0.000000
2,0,15720,False,3.0,0.025641,0.000514,0.009709
3,0,8056,False,6.0,0.020000,0.001886,0.000000
4,0,2843,False,6.0,0.008621,0.003038,0.020840
...,...,...,...,...,...,...,...
631895,12637,13251,False,9.0,0.011597,0.031139,0.225243
631896,12637,13252,False,8.0,0.004134,0.041050,0.425408
631897,12637,10900,False,19.0,0.004039,0.044949,0.475032
631898,12637,10427,False,6.0,0.009028,0.024508,0.164902


In [12]:
item_popularity = np.ediff1d(sps.csc_matrix(data_train_val).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train_val).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
training_dataframe

,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF,item_popularity,user_profile_len
0,0,15344,False,4.0,0.029524,0.003029,0.019198,4,42
1,0,20178,False,3.0,0.010204,0.000170,0.000000,3,42
2,0,15720,False,3.0,0.025641,0.000514,0.009709,3,42
3,0,8056,False,6.0,0.020000,0.001886,0.000000,6,42
4,0,2843,False,6.0,0.008621,0.003038,0.020840,6,42
...,...,...,...,...,...,...,...,...,...
631895,12637,13251,False,9.0,0.011597,0.031139,0.225243,9,63
631896,12637,13252,False,8.0,0.004134,0.041050,0.425408,8,63
631897,12637,10900,False,19.0,0.004039,0.044949,0.475032,19,63
631898,12637,10427,False,6.0,0.009028,0.024508,0.164902,6,63


In [13]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns = ['index'], inplace=True)
training_dataframe

,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF,item_popularity,user_profile_len
0,0,15344,False,4.0,0.029524,0.003029,0.019198,4,42
1,0,3708,False,163.0,0.000000,0.056358,0.378622,163,42
2,0,731,False,37.0,0.008714,0.018729,0.208664,37,42
3,0,19566,False,2.0,0.000000,0.000732,0.000000,2,42
4,0,970,False,16.0,0.016829,0.009553,0.097634,16,42
...,...,...,...,...,...,...,...,...,...
631895,12637,16596,False,1.0,0.000000,0.010591,0.038972,1,63
631896,12637,564,False,106.0,0.018380,0.133407,0.990308,106,63
631897,12637,19608,False,3.0,0.006570,0.005879,0.000000,3,63
631898,12637,10905,False,12.0,0.027636,0.041590,0.456375,12,63


In [14]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [15]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-02 22:21:51,660] A new study created in memory with name: XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)


In [16]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [20]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_recs,
        train_labs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train_val, train_recs, XGB_model, training_dataframe, data_test)
    
    return ev_map

study.optimize(objective, n_trials=1000)

#Trial 6 finished with value: 0.12485196495596997 and parameters: {'objective': 'rank:ndcg', 'n_estimators': 336, 'learning_rate': 0.023887567768263025, 'reg_alpha': 0.01495634916569455, 'reg_lambda': 0.004873292863879341, 'max_depth': 4, 'max_leaves': 3, 'grow_policy': 'depthwise', 'booster': 'gbtree'}. Best is trial 6 with value: 0.12485196495596997.

[W 2024-01-02 22:27:36,458] Trial 3 failed with parameters: {'objective': 'rank:ndcg', 'n_estimators': 164, 'learning_rate': 0.08341482662047495, 'reg_alpha': 0.00021445168785073766, 'reg_lambda': 0.0011741446495147766, 'max_depth': 2, 'max_leaves': 5, 'grow_policy': 'lossguide', 'booster': 'dart'} because of the following error: ValueError('DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:ItemID: object').
Traceback (most recent call last):
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Francesco1\AppData\Local\Temp\ipykernel_16968\3806412458.py", line 28, in objective
    XGB_model.fit(
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\xgboost\core.py", line 730, in inner_f
    retu

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:ItemID: object

In [ ]:
study.best_params

{'objective': 'rank:map',
 'n_estimators': 161,
 'learning_rate': 0.00013256002295690622,
 'reg_alpha': 0.008487026488935363,
 'reg_lambda': 0.0029756545632511457,
 'max_depth': 6,
 'max_leaves': 8,
 'grow_policy': 'lossguide',
 'booster': 'gbtree'}

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
    enable_categorical=True,
)
XGB_model.fit(
    train_recs,
    train_labs,
    group=groups,
    verbose=False
)

#XGB_model = XGBRanker(
#    objective='rank:ndcg', 
#    n_estimators=336, 
#    learning_rate=0.023887567768263025, 
#    reg_alpha=0.01495634916569455, 
#    reg_lambda=0.004873292863879341, 
#    max_depth=4, 
#    max_leaves=3, 
#    grow_policy= 'depthwise', 
#    booster='gbtree',
#    ndcg_exp_gain=False,
#    verbosity=0, # 2 if self.verbose else 0,
#    enable_categorical=True,
#)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None,
          device=None, early_stopping_rounds=None, enable_categorical=True,
          eval_metric=None, feature_types=None, gamma=None,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints=None, learning_rate=0.023887567768263025,
          max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
          max_delta_step=None, max_depth=4, max_leaves=3, min_child_weight=None,
          missing=nan, monotone_constraints=None, multi_strategy=None,
          n_estimators=336, n_jobs=None, ndcg_exp_gain=False,
          num_parallel_tree=None, ...)

In [ ]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)

TopPopRecommender: URM Detected 212 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 108 ( 0.5%) items with no interactions.
